In [3]:
import pandas as pd
import time
from datetime import datetime
import ast
import numpy as np

In [4]:
csv_file = 'Big_five_2021-2022_merged.csv'
columns_to_check = ['Home', 'Away', 'Date']  

df_base = pd.read_csv(csv_file, encoding='utf-8')
df_base = df_base.apply(lambda x: x.map(lambda y: y.replace('\xa0', ' ') if isinstance(y, str) else y))
duplicateRows = df_base[df_base.duplicated()]
df = df_base

df

,League,Home,Away,Home Score,Away Score,Home xG,Away xG,Home League Score,Away League Score,Home Scorers,...,Home Goal Kicks,Away Goal Kicks,Home Throw Ins,Away Throw Ins,Home Long Balls,Away Long Balls,Home Yellow Cards,Home Red Cards,Away Yellow Cards,Away Red Cards
0,Bundesliga,Mönchengladbach,Bayern Munich,1,1,2.3,2.5,0-1-0,0-1-0,['Alassane Pléa'],...,9,6,19,29,79,63,2,0,0,0
1,Bundesliga,Stuttgart,Greuther Fürth,5,1,3.0,1.2,1-0-0,0-0-1,"['Wataru Endo', 'Philipp Klement', 'Marc-Olive...",...,4,9,21,13,65,60,0,0,1,0
2,Bundesliga,Union Berlin,Bayer Leverkusen,1,1,0.5,0.8,0-1-0,0-1-0,['Taiwo Awoniyi'],...,6,8,25,25,66,69,0,0,1,0
3,Bundesliga,Wolfsburg,Bochum,1,0,2.0,0.4,1-0-0,0-0-1,['Wout Weghorst'],...,6,10,25,34,69,88,0,0,2,1
4,Bundesliga,Arminia,Freiburg,0,0,0.4,1.6,0-1-0,0-1-0,[],...,12,7,27,17,105,84,3,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,Serie A,Venezia,Cagliari,0,0,1.0,1.9,6/9/2023,6/12/2020,[],...,14,9,15,13,66,69,0,0,3,0
1821,Serie A,Salernitana,Udinese,0,4,1.6,2.5,7/10/2021,11/14/2013,[],...,6,10,10,9,61,46,1,0,1,0
1822,Serie A,Cagliari,Venezia,1,1,0.5,0.7,0-3-4,1/2/2004,['Keita Baldé'],...,6,10,19,30,79,71,2,0,5,0
1823,Serie A,Spezia,Sassuolo,2,2,2.5,1.5,3/3/2010,5/5/2006,"['Rey Manaj', 'Emmanuel Gyasi']",...,9,3,18,16,62,56,4,0,3,0


In [5]:
df.isnull().sum()

League               0
Home                 0
Away                 0
Home Score           0
Away Score           0
                    ..
Away Long Balls      0
Home Yellow Cards    0
Home Red Cards       0
Away Yellow Cards    0
Away Red Cards       0
Length: 68, dtype: int64

In [4]:
df['Away Captain']

0             Manuel Neuer
1          Branimir Hrgota
2           Lukáš Hrádecký
3          Anthony Losilla
4         Christian Günter
               ...        
1820            João Pedro
1821       Roberto Pereyra
1822      Pietro Ceccaroni
1823    Gian Marco Ferrari
1824      Gianluca Mancini
Name: Away Captain, Length: 1825, dtype: object

In [6]:
df.columns

Index(['League', 'Home', 'Away', 'Home Score', 'Away Score', 'Home xG',
       'Away xG', 'Home League Score', 'Away League Score', 'Home Scorers',
       'Away Scorers', 'Home Penalty Scorers', 'Away Penalty Scorers',
       'Home Own Goal Scorers', 'Away Own Goal Scorers', 'Home Goals Minutes',
       'Away Goals Minutes', 'Home Penalty Goals Minutes',
       'Away Penalty Goals Minutes', 'Home Own Goal Minutes',
       'Away Own Goal Minutes', 'Home Manager', 'Away Manager', 'Home Captain',
       'Away Captain', 'Date', 'Venue Time', 'Attendance', 'Venue', 'Referee',
       'AR1', 'AR2', 'Fourth Official', 'VAR', 'Home Formation',
       'Away Formation', 'Home First Squad', 'Away First Squad', 'Home Bench',
       'Away Bench', 'Home Fouls', 'Away Fouls', 'Home Corners',
       'Away Corners', 'Home Crosses', 'Away Crosses', 'Home Touches',
       'Away Touches', 'Home Tackles', 'Away Tackles', 'Home Interceptions',
       'Away Interceptions', 'Home Aerials Won', 'Away Aerials Wo

In [6]:
league_names = df['League'].unique().tolist()
leagues_dict = {i+1: league for i, league in enumerate(league_names)}
df['League_id'] = df['League'].map({v: k for k, v in leagues_dict.items()})


In [7]:
#1 (Legues SQL insert statement)
values_clause = ", ".join([f"({league_id}, '{league_name}')" for league_id, league_name in leagues_dict.items()])
sql_insert_laegue = f"INSERT INTO League (LeagueID, LeagueName) VALUES {values_clause};"

sql_insert_laegue

"INSERT INTO League (LeagueID, LeagueName) VALUES (1, 'Bundesliga'), (2, 'LaLiga'), (3, 'Ligue 1'), (4, 'Premier League'), (5, 'Serie A');"

In [8]:
home_teams = df["Home"].unique()
away_teams = df["Away"].unique()

teams_dict = {}

for league_id, league_name in leagues_dict.items():
    # Filter DataFrame for each league
    league_df = df[df['League'] == league_name]
    
    # Extract home and away teams for the current league
    home_teams = league_df['Home'].unique()
    away_teams = league_df['Away'].unique()
    
    # Populate teams_dict with the correct league ID for each team
    teams_dict.update({team: league_id for team in home_teams})
    teams_dict.update({team: league_id for team in away_teams})

# Enumerate from 1 as keys for teams
teams_enum_dict = {i+1: (team, league_id) for i, (team, league_id) in enumerate(teams_dict.items())}
teams_enum_dict

{1: ('Mönchengladbach', 1),
 2: ('Stuttgart', 1),
 3: ('Union Berlin', 1),
 4: ('Wolfsburg', 1),
 5: ('Arminia', 1),
 6: ('Augsburg', 1),
 7: ('Dortmund', 1),
 8: ('Mainz 05', 1),
 9: ('Köln', 1),
 10: ('RB Leipzig', 1),
 11: ('Hertha BSC', 1),
 12: ('Bochum', 1),
 13: ('Greuther Fürth', 1),
 14: ('Freiburg', 1),
 15: ('Eintracht Frankfurt', 1),
 16: ('Bayer Leverkusen', 1),
 17: ('Bayern Munich', 1),
 18: ('Hoffenheim', 1),
 19: ('Valencia', 2),
 20: ('Mallorca', 2),
 21: ('Cádiz', 2),
 22: ('Alavés', 2),
 23: ('Osasuna', 2),
 24: ('Celta Vigo', 2),
 25: ('Barcelona', 2),
 26: ('Sevilla', 2),
 27: ('Villarreal', 2),
 28: ('Elche', 2),
 29: ('Real Betis', 2),
 30: ('Espanyol', 2),
 31: ('Granada', 2),
 32: ('Athletic Club', 2),
 33: ('Real Sociedad', 2),
 34: ('Atlético Madrid', 2),
 35: ('Levante', 2),
 36: ('Getafe', 2),
 37: ('Rayo Vallecano', 2),
 38: ('Real Madrid', 2),
 39: ('Monaco', 3),
 40: ('Lyon', 3),
 41: ('Troyes', 3),
 42: ('Rennes', 3),
 43: ('Bordeaux', 3),
 44: ('Stras

In [9]:
# 2 SQL INSERT statement for teams
values_clause_teams = ", ".join([f"({team_id}, {league_id}, '{team_name}')" for team_id, (team_name, league_id) in teams_enum_dict.items()])
sql_insert_teams = f"INSERT INTO Team(TeamID, LeagueID, TeamName) VALUES {values_clause_teams};"
sql_insert_teams

"INSERT INTO Team(TeamID, LeagueID, TeamName) VALUES (1, 1, 'Mönchengladbach'), (2, 1, 'Stuttgart'), (3, 1, 'Union Berlin'), (4, 1, 'Wolfsburg'), (5, 1, 'Arminia'), (6, 1, 'Augsburg'), (7, 1, 'Dortmund'), (8, 1, 'Mainz 05'), (9, 1, 'Köln'), (10, 1, 'RB Leipzig'), (11, 1, 'Hertha BSC'), (12, 1, 'Bochum'), (13, 1, 'Greuther Fürth'), (14, 1, 'Freiburg'), (15, 1, 'Eintracht Frankfurt'), (16, 1, 'Bayer Leverkusen'), (17, 1, 'Bayern Munich'), (18, 1, 'Hoffenheim'), (19, 2, 'Valencia'), (20, 2, 'Mallorca'), (21, 2, 'Cádiz'), (22, 2, 'Alavés'), (23, 2, 'Osasuna'), (24, 2, 'Celta Vigo'), (25, 2, 'Barcelona'), (26, 2, 'Sevilla'), (27, 2, 'Villarreal'), (28, 2, 'Elche'), (29, 2, 'Real Betis'), (30, 2, 'Espanyol'), (31, 2, 'Granada'), (32, 2, 'Athletic Club'), (33, 2, 'Real Sociedad'), (34, 2, 'Atlético Madrid'), (35, 2, 'Levante'), (36, 2, 'Getafe'), (37, 2, 'Rayo Vallecano'), (38, 2, 'Real Madrid'), (39, 3, 'Monaco'), (40, 3, 'Lyon'), (41, 3, 'Troyes'), (42, 3, 'Rennes'), (43, 3, 'Bordeaux'), (4

In [10]:
# 3 Extract home and away managers with corresponding teams
home_managers = df["Home Manager"].unique()
away_managers = df["Away Manager"].unique()


# Create a dictionary with managers associated with team IDs
managers_dict = {manager: team_id for team_id, (team, league_id) in teams_enum_dict.items() for manager in [df.loc[df["Home"] == team, "Home Manager"].iloc[0], df.loc[df["Away"] == team, "Away Manager"].iloc[0]]}

# Create a single SQL INSERT statement for managers
values_clause_managers = ", ".join([f"""({team_id}, {team_id}, "{manager}")""" for manager, team_id in managers_dict.items()])
sql_insert_managers = f"INSERT INTO Manager(ManagerID,TeamID,ManagerName) VALUES {values_clause_managers};"
sql_insert_managers

'INSERT INTO Manager(ManagerID,TeamID,ManagerName) VALUES (1, 1, "Adi Hütter"), (2, 2, "Pellegrino Matarazzo"), (3, 3, "Urs Fischer"), (4, 4, "Mark van Bommel"), (5, 5, "Frank Kramer"), (6, 6, "Markus Weinzierl"), (7, 7, "Marco Rose"), (8, 8, "Bo Svensson"), (9, 9, "Steffen Baumgart"), (10, 10, "Jesse Marsch"), (11, 11, "Pál Dárdai"), (12, 12, "Thomas Reis"), (13, 13, "Stefan Leitl"), (14, 14, "Christian Streich"), (15, 15, "Oliver Glasner"), (16, 16, "Gerardo Seoane"), (17, 17, "Julian Nagelsmann"), (18, 18, "Sebastian Hoeneß"), (19, 19, "Pepe Bordalás"), (20, 20, "Luis García"), (21, 21, "Álvaro Cervera"), (22, 22, "Javier Calleja"), (23, 23, "Jagoba Arrasate"), (24, 24, "Eduardo Coudet"), (25, 25, "Ronald Koeman"), (26, 26, "Lopetegui"), (27, 27, "Unai Emery"), (28, 28, "Fran Escribá"), (29, 29, "Manuel Pellegrini"), (30, 30, "Vicente Moreno"), (31, 31, "Robert Moreno"), (32, 32, "Marcelino"), (33, 33, "Imanol Alguacil"), (34, 34, "Diego Simeone"), (35, 35, "Paco López"), (36, 36, "

In [11]:
# 4 Extract home and away captains with corresponding teams
home_captains = df["Home Captain"].unique()
away_captains = df["Away Captain"].unique()

captains_dict = {}
for team_id, (team, league_id) in teams_enum_dict.items():
    home_captains_for_team = df.loc[df["Home"] == team, "Home Captain"].unique()
    away_captains_for_team = df.loc[df["Away"] == team, "Away Captain"].unique()
    for captain in np.concatenate((home_captains_for_team, away_captains_for_team)):
        captains_dict[captain] = team_id
captains_enum_dict = {i+1: (captain, team_id) for i, (captain, team_id) in enumerate(captains_dict.items())}
captains_insert_statements = []

values_clause_captains = ", ".join([f"""({captain_id}, {team_id}, "{captain_name}")""" for captain_id, (captain_name, team_id) in captains_enum_dict.items()])
sql_insert_captains = f"INSERT INTO Captain(CaptainID, TeamID, CaptainName) VALUES {values_clause_captains};"
sql_insert_captains

'INSERT INTO Captain(CaptainID, TeamID, CaptainName) VALUES (1, 1, "Lars Stindl"), (2, 1, "Yann Sommer"), (3, 2, "Wataru Endo"), (4, 2, "Hiroki Ito"), (5, 2, "Waldemar Anton"), (6, 3, "Christopher Trimmel"), (7, 3, "Marvin Friedrich"), (8, 3, "Grischa Prömel"), (9, 3, "Rani Khedira"), (10, 4, "Koen Casteels"), (11, 4, "Maximilian Arnold"), (12, 5, "Manuel Prietl"), (13, 5, "Cédric Brunner"), (14, 5, "Stefan Ortega"), (15, 5, "Fabian Klos"), (16, 6, "Jeffrey Gouweleeuw"), (17, 6, "Ohis Felix Uduokhai"), (18, 6, "André Hahn"), (19, 7, "Marco Reus"), (20, 7, "Axel Witsel"), (21, 7, "Mats Hummels"), (22, 8, "Moussa Niakhate"), (23, 8, "Robin Zentner"), (24, 9, "Jonas Hector"), (25, 9, "Anthony Modeste"), (26, 9, "Rafael Czichos"), (27, 10, "Emil Forsberg"), (28, 10, "Péter Gulácsi"), (29, 10, "Kevin Kampl"), (30, 10, "Willi Orban"), (31, 11, "Dedryck Boyata"), (32, 11, "Niklas Stark"), (33, 11, "Peter Pekarík"), (34, 12, "Anthony Losilla"), (35, 12, "Manuel Riemann"), (36, 13, "Branimir Hr

In [12]:
# 5 Extract unique formations
formations = pd.concat([df['Home Formation'], df['Away Formation']]).unique()
formations_dict = {i+1: formation for i, formation in enumerate(formations)}
values_clause_formations = ", ".join([f"({formation_id}, '{formation_name}')" for formation_id, formation_name in formations_dict.items()])
sql_insert_formations = f"INSERT INTO Formation(FormationID, FormationName) VALUES {values_clause_formations};"
sql_insert_formations


"INSERT INTO Formation(FormationID, FormationName) VALUES (1, '4-2-3-1'), (2, '3-4-3'), (3, '5-3-2'), (4, '4-1-2-1-2'), (5, '3-4-1-2'), (6, '4-4-2'), (7, '3-5-2'), (8, '4-1-3-2'), (9, '4-3-3'), (10, '4-2-2-2'), (11, '3-1-4-2'), (12, '4-3-1-2'), (13, '4-4-1-1'), (14, '4-1-4-1'), (15, '5-4-1'), (16, '4-5-1'), (17, '4-2-4-0'), (18, '3-2-4-1'), (19, '3-5-1-1'), (20, '3-3-3-1'), (21, '4-3-2-1');"

In [13]:
# 6 Referees
referees = df['Referee'].unique()
referees_dict = {i+1: referee for i, referee in enumerate(referees)}
values_clause_referees = ", ".join([f"""({referee_id}, "{referee_name}")""" for referee_id, referee_name in referees_dict.items()])
sql_insert_referees = f"INSERT INTO Referee(RefereeID, RefereeName) VALUES {values_clause_referees};"
sql_insert_referees

'INSERT INTO Referee(RefereeID, RefereeName) VALUES (1, "Marco Fritz"), (2, "Felix Zwayer"), (3, "Tobias Reichel"), (4, "Frank Willenborg"), (5, "Sven Jablonski"), (6, "Felix Brych"), (7, "Tobias Stieler"), (8, "Daniel Siebert"), (9, "Robert Hartmann"), (10, "Florian Badstübner"), (11, "Matthias Jöllenbeck"), (12, "Patrick Ittrich"), (13, "Daniel Schlager"), (14, "Robert Schröder"), (15, "Harm Osmers"), (16, "Deniz Aytekin"), (17, "Martin Petersen"), (18, "Bastian Dankert"), (19, "Benjamin Brand"), (20, "Benjamin Cortus"), (21, "Sascha Stegemann"), (22, "Tobias Welz"), (23, "Christian Dingert"), (24, "Robert Schörgenhofer"), (25, "Jesús Gil"), (26, "Antonio Matéu Lahoz"), (27, "Santiago Jaime"), (28, "César Soto"), (29, "Miguel Ángel Ortiz Arias"), (30, "José Luis Munuera"), (31, "Alejandro Hernández"), (32, "Isidro Díaz de Mera"), (33, "Pablo González"), (34, "Adrián Cordero"), (35, "Valentín Pizarro"), (36, "Alejandro Muñíz"), (37, "Jorge Figueroa"), (38, "Carlos del Cerro"), (39, "J

In [14]:
# 7 Extract unique AR1 values
ar1_values = df['AR1'].unique()
ar1_dict = {i+1: ar1_value for i, ar1_value in enumerate(ar1_values)}
values_clause_ar1 = ", ".join([f"""({ar1_id}, "{ar1_value}")""" for ar1_id, ar1_value in ar1_dict.items()])
sql_insert_ar1 = f"INSERT INTO AR1(AR1ID, AR1Name) VALUES {values_clause_ar1};"
sql_insert_ar1

'INSERT INTO AR1(AR1ID, AR1Name) VALUES (1, "Christof Günsch"), (2, "Rafael Foltyn"), (3, "Benedikt Kempkes"), (4, "Guido Kleve"), (5, "Norbert Grudzinski"), (6, "Mark Borsch"), (7, "Christian Gittelmann"), (8, "Lasse Koslowski"), (9, "Christian Leicher"), (10, "Markus Schüller"), (11, "Dominik Schaal"), (12, "Sascha Thielert"), (13, "Sven Waschitzki"), (14, "Jan Neitzel"), (15, "Robert Kempter"), (16, "Christian Dietz"), (17, "Alexander Sather"), (18, "Marco Achmüller"), (19, "Christian Bandurski"), (20, "René Rohde"), (21, "Florian Heft"), (22, "Stefan Lupp"), (23, "Mike Pickel"), (24, "Eduard Beitinger"), (25, "Martin Thomsen"), (26, "Frederick Assmuth"), (27, "Marcel Unger"), (28, "Jan Seidel"), (29, "Aarne Aarnink"), (30, "Thomas Stein"), (31, "Ángel Nevado"), (32, "Pau Cebrián Devis"), (33, "Jorge Bueno"), (34, "Carlos Álvarez"), (35, "Alejandro Estevez Iglesias"), (36, "Iñigo Prieto"), (37, "José Naranjo"), (38, "Rubén Becerril"), (39, "Alfredo Rodríguez"), (40, "Raúl Cabañero")

In [15]:
# 8, 9 ,10 Extract unique values for AR2, FourthOfficial, and VAR
ar2_values = df['AR2'].unique()
fourth_official_values = df['Fourth Official'].unique()
var_values = df['VAR'].unique()

ar2_dict = {i+1: ar2_value for i, ar2_value in enumerate(ar2_values)}
fourth_official_dict = {i+1: fourth_official_value for i, fourth_official_value in enumerate(fourth_official_values)}
var_dict = {i+1: var_value for i, var_value in enumerate(var_values)}

values_clause_ar2 = ", ".join([f"""({ar2_id}, "{ar2_value}")""" for ar2_id, ar2_value in ar2_dict.items()])
sql_insert_ar2 = f"INSERT INTO AR2(AR2ID, AR2Name) VALUES {values_clause_ar2};"

values_clause_fourth_official = ", ".join([f"""({fourth_official_id}, "{fourth_official_value}")""" for fourth_official_id, fourth_official_value in fourth_official_dict.items()])
sql_insert_fourth_official = f"INSERT INTO FourthOfficial(FourthOfficialID, FourthOfficialName) VALUES {values_clause_fourth_official};"

values_clause_var = ", ".join([f"""({var_id}, "{var_value}")""" for var_id, var_value in var_dict.items()])
sql_insert_var = f"INSERT INTO VAR(VARID, VARName) VALUES {values_clause_var};"

print(sql_insert_ar2)
print(sql_insert_fourth_official)
print(sql_insert_var)


INSERT INTO AR2(AR2ID, AR2Name) VALUES (1, "Marcel Pelgrim"), (2, "René Rohde"), (3, "Timo Gerach"), (4, "Marcel Unger"), (5, "Sascha Thielert"), (6, "Stefan Lupp"), (7, "Eduard Beitinger"), (8, "Martin Thomsen"), (9, "Arno Blos"), (10, "Philipp Hüwe"), (11, "Jonas Weickenmeier"), (12, "Christian Bandurski"), (13, "Thomas Gorniak"), (14, "Markus Sinn"), (15, "Robert Wessel"), (16, "Aarne Aarnink"), (17, "Mark Borsch"), (18, "Thorben Siewer"), (19, "Robert Kempter"), (20, "Rafael Foltyn"), (21, "Markus Schüller"), (22, "Frederick Assmuth"), (23, "Benedikt Kempkes"), (24, "Jan Neitzel"), (25, "Christian Gittelmann"), (26, "Guido Kleve"), (27, "Jan Seidel"), (28, "Dominik Schaal"), (29, "Norbert Grudzinski"), (30, "Marco Achmüller"), (31, "Mike Pickel"), (32, "Christian Dietz"), (33, "Lasse Koslowski"), (34, "Christian Leicher"), (35, "Thomas Stein"), (36, "Javier Martínez"), (37, "Rubén Porras"), (38, "Fernando Tresaco Escabosa"), (39, "Julián Villaseñor"), (40, "Iker De Francisco"), (41

In [16]:
# 11 Extract unique venues
venues = df['Venue'].unique()
venues_dict = {i+1: venue for i, venue in enumerate(venues)}
values_clause_venues = ", ".join([f"""({venue_id}, "{venue_name}")""" for venue_id, venue_name in venues_dict.items()])
sql_insert_venues = f"INSERT INTO Venue(VenueID, VenueName) VALUES {values_clause_venues};"
sql_insert_venues

'INSERT INTO Venue(VenueID, VenueName) VALUES (1, "Stadion im Borussia-Park, Mönchengladbach"), (2, "Mercedes-Benz Arena, Stuttgart"), (3, "Stadion An der Alten Försterei, Berlin"), (4, "Volkswagen Arena, Wolfsburg"), (5, "SchücoArena, Bielefeld"), (6, "WWK Arena, Augsburg"), (7, "Signal Iduna Park, Dortmund"), (8, "Mewa Arena, Mainz"), (9, "RheinEnergieSTADION, Köln"), (10, "Red Bull Arena, Leipzig"), (11, "Olympiastadion Berlin, Berlin"), (12, "Vonovia Ruhrstadion, Bochum"), (13, "Sportpark Ronhof Thomas Sommer, Fürth"), (14, "Schwarzwald-Stadion, Freiburg"), (15, "Deutsche Bank Park, Frankfurt am Main"), (16, "BayArena, Leverkusen"), (17, "Allianz Arena, München"), (18, "PreZero Arena, Sinsheim"), (19, "Europa-Park Stadion, Freiburg"), (20, "Estadio de Mestalla, Valencia"), (21, "Iberostar Estadi, Palma de Mallorca"), (22, "Estadio Nuevo Mirandilla, Cádiz"), (23, "Estadio de Mendizorroza, Vitoria-Gasteiz"), (24, "Estadio El Sadar, Pamplona"), (25, "Estadio de Balaídos, Vigo"), (26, 

In [17]:
# 12  date in the format 'Friday August 13, 2021'

match_dates = df['Date'].unique()
match_dates_dict = {i+1: match_date for i, match_date in enumerate(match_dates)}

values_clause_match_dates = ", ".join([f"({match_date_id}, '{match_date}')" for match_date_id, match_date in match_dates_dict.items()])
sql_insert_match_dates = f"INSERT INTO MatchDate(MatchDateID, MatchDate) VALUES {values_clause_match_dates};"
sql_insert_match_dates

"INSERT INTO MatchDate(MatchDateID, MatchDate) VALUES (1, '2021-08-13'), (2, '2021-08-14'), (3, '2021-08-15'), (4, '2021-08-20'), (5, '2021-08-21'), (6, '2021-08-22'), (7, '2021-08-27'), (8, '2021-08-28'), (9, '2021-08-29'), (10, '2021-09-11'), (11, '2021-09-12'), (12, '2021-09-17'), (13, '2021-09-18'), (14, '2021-09-19'), (15, '2021-09-24'), (16, '2021-09-25'), (17, '2021-09-26'), (18, '2021-10-01'), (19, '2021-10-02'), (20, '2021-10-03'), (21, '2021-10-15'), (22, '2021-10-16'), (23, '2021-10-17'), (24, '2021-10-22'), (25, '2021-10-23'), (26, '2021-10-24'), (27, '2021-10-29'), (28, '2021-10-30'), (29, '2021-10-31'), (30, '2021-11-05'), (31, '2021-11-06'), (32, '2021-11-07'), (33, '2021-11-19'), (34, '2021-11-20'), (35, '2021-11-21'), (36, '2021-11-26'), (37, '2021-11-27'), (38, '2021-11-28'), (39, '2021-12-03'), (40, '2021-12-04'), (41, '2021-12-05'), (42, '2021-12-10'), (43, '2021-12-11'), (44, '2021-12-12'), (45, '2021-12-14'), (46, '2021-12-15'), (47, '2021-12-17'), (48, '2021-12-1

In [18]:
# Extract data for FootballMatch table
football_match_data = df[[
    'League', 'Home', 'Away', 'Home Score', 'Away Score', 'Home xG', 'Away xG',
    'Date', 'Venue Time', 'Venue'
]]

leagues_dict_matchdate = {v: k for k, v in leagues_dict.items()}
teams_ID_dict = {key: value[0] for key, value in teams_enum_dict.items()}
teams_ID_dict_matchdate = {v: k for k, v in teams_ID_dict.items()}
venues_dict_reverse = {v: k for k, v in venues_dict.items()}
match_dates_dict_revers = {v: k for k, v in match_dates_dict.items()}

football_match_data.loc[:, 'League'] = football_match_data['League'].map(leagues_dict_matchdate)
football_match_data.loc[:, 'Home'] = football_match_data['Home'].map(teams_ID_dict_matchdate)
football_match_data.loc[:, 'Away'] = football_match_data['Away'].map(teams_ID_dict_matchdate)
football_match_data.loc[:, 'Venue'] = football_match_data['Venue'].map(venues_dict_reverse)
football_match_data.loc[:, 'Date'] = football_match_data['Date'].map(match_dates_dict_revers)

print(football_match_data)

     League Home Away  Home Score  Away Score  Home xG  Away xG Date  \
0         1    1   17           1           1      2.3      2.5    1   
1         1    2   13           5           1      3.0      1.2    2   
2         1    3   16           1           1      0.5      0.8    2   
3         1    4   12           1           0      2.0      0.4    2   
4         1    5   14           0           0      0.4      1.6    2   
...     ...  ...  ...         ...         ...      ...      ...  ...   
1820      5   98   87           0           0      1.0      1.9  151   
1821      5   96   84           0           4      1.6      2.5  151   
1822      5   87   98           1           1      0.5      0.7   18   
1823      5   97   93           2           2      2.5      1.5   41   
1824      5   82   86           2           4      0.9      2.5   58   

     Venue Time Venue  
0         20:30     1  
1         15:30     2  
2         15:30     3  
3         15:30     4  
4         15:30

In [19]:
sql_statements = []

for index, row in football_match_data.iterrows():
    match_id = index 
    league_id = row['League']
    home_team_id = row['Home']
    away_team_id = row['Away']
    home_score = row['Home Score']
    away_score = row['Away Score']
    home_xg = row['Home xG']
    away_xg = row['Away xG']
    match_date_id = row['Date']
    match_time = row['Venue Time']
    venue_id = row['Venue']
    values_part = f"({match_id+1}, {league_id}, {home_team_id}, {away_team_id}, {home_score}, {away_score}, {home_xg}, {away_xg}, {match_date_id}, '{match_time}', {venue_id})"
    sql_statements.append(values_part)

column_names = "MatchID, LeagueID, HomeTeamID, AwayTeamID, HomeScore, AwayScore, HomeXG, AwayXG, MatchDateID, MatchTime, VenueID"
final_sql_statement_matchdata = f"INSERT INTO FootballMatch({column_names}) VALUES {', '.join(sql_statements)};"

print(final_sql_statement_matchdata)

INSERT INTO FootballMatch(MatchID, LeagueID, HomeTeamID, AwayTeamID, HomeScore, AwayScore, HomeXG, AwayXG, MatchDateID, MatchTime, VenueID) VALUES (1, 1, 1, 17, 1, 1, 2.3, 2.5, 1, '20:30', 1), (2, 1, 2, 13, 5, 1, 3.0, 1.2, 2, '15:30', 2), (3, 1, 3, 16, 1, 1, 0.5, 0.8, 2, '15:30', 3), (4, 1, 4, 12, 1, 0, 2.0, 0.4, 2, '15:30', 4), (5, 1, 5, 14, 0, 0, 0.4, 1.6, 2, '15:30', 5), (6, 1, 6, 18, 0, 4, 0.6, 3.8, 2, '15:30', 6), (7, 1, 7, 15, 5, 2, 2.7, 1.3, 2, '18:30', 7), (8, 1, 8, 10, 1, 0, 1.2, 1.1, 3, '15:30', 8), (9, 1, 9, 11, 3, 1, 1.6, 1.8, 3, '17:30', 9), (10, 1, 10, 2, 4, 0, 3.6, 1.1, 4, '20:30', 10), (11, 1, 11, 4, 1, 2, 1.2, 1.0, 5, '15:30', 11), (12, 1, 12, 8, 2, 0, 0.8, 0.9, 5, '15:30', 12), (13, 1, 13, 5, 1, 1, 2.5, 0.8, 5, '15:30', 13), (14, 1, 14, 7, 2, 1, 1.3, 1.6, 5, '15:30', 14), (15, 1, 15, 6, 0, 0, 1.6, 0.3, 5, '15:30', 15), (16, 1, 16, 1, 4, 0, 1.9, 1.7, 5, '18:30', 16), (17, 1, 17, 9, 3, 2, 2.6, 1.1, 6, '17:30', 17), (18, 1, 7, 18, 3, 2, 3.4, 2.2, 7, '20:30', 7), (19, 1, 

In [20]:
# Extract data for Scorer table
scorer_data = df[['Home', 'Away', 'Home First Squad','Away First Squad','Home Bench','Away Bench']]

player_teams = {}

for index, match in scorer_data.iterrows():
    home_team = match['Home']
    away_team = match['Away']

    # Home First Squad
    for player in eval(match['Home First Squad']):
        player_teams[player] = home_team

    # Away First Squad
    for player in eval(match['Away First Squad']):
        player_teams[player] = away_team

    # Home Bench
    for player in eval(match['Home Bench']):
        player_teams[player] = home_team

    # Away Bench
    for player in eval(match['Away Bench']):
        player_teams[player] = away_team


team_names_to_numbers = {team[0]: number for number, team in teams_enum_dict.items()}
player_teams_swapped = {player: team_names_to_numbers[team] for player, team in player_teams.items()}

list_of_lists = []

for index, (key, value) in enumerate(player_teams_swapped.items(), 1):
    inner_list = [index, {key: value}]
    list_of_lists.append(inner_list)

values = ", ".join([f"""({entry[0]}, {list(entry[1].values())[0]}, "{list(entry[1].keys())[0]}")""" for entry in list_of_lists])

insert_statement_players = f"INSERT INTO Player (PlayerID, TeamID, PlayerName) VALUES {values};"

print(insert_statement_players)


INSERT INTO Player (PlayerID, TeamID, PlayerName) VALUES (1, 1, "Yann Sommer"), (2, 1, "Christoph Kramer"), (3, 1, "Patrick Herrmann"), (4, 1, "Hannes Wolf"), (5, 1, "Lars Stindl"), (6, 1, "Alassane Pléa"), (7, 1, "Stefan Lainer"), (8, 1, "Matthias Ginter"), (9, 1, "Joe Scally"), (10, 1, "Nico Elvedi"), (11, 1, "Florian Neuhaus"), (12, 17, "Manuel Neuer"), (13, 17, "Dayot Upamecano"), (14, 17, "Niklas Süle"), (15, 17, "Joshua Kimmich"), (16, 17, "Serge Gnabry"), (17, 17, "Leon Goretzka"), (18, 17, "Robert Lewandowski"), (19, 17, "Leroy Sané"), (20, 17, "Alphonso Davies"), (21, 17, "Thomas Müller"), (22, 17, "Josip Stanišić"), (23, 1, "Tobias Sippel"), (24, 1, "Marcus Thuram"), (25, 1, "Jordan Beyer"), (26, 1, "Luca Netz"), (27, 1, "László Bénes"), (28, 1, "Jonas Hofmann"), (29, 1, "Tony Jantschke"), (30, 1, "Connor Noß"), (31, 1, "Keanan Bennetts"), (32, 17, "Sven Ulreich"), (33, 17, "Omar Richards"), (34, 17, "Kingsley Coman"), (35, 17, "Eric Maxim Choupo-Moting"), (36, 18, "Chris Ric

In [21]:
# scorer_data 
scorer_list_of_lists = []

for index, row in scorer_data.iterrows():
    home_team = row['Home']
    away_team = row['Away']
    home_first_squad = eval(row['Home First Squad'])
    away_first_squad = eval(row['Away First Squad'])
    home_bench = eval(row['Home Bench'])
    away_bench = eval(row['Away Bench'])
    
    row_id = index + 1

    # Create a list for each player in the home first squad
    for player in home_first_squad:
        scorer_list = [player, home_team, row_id, True]
        scorer_list_of_lists.append(scorer_list)

    # Create a list for each player in the away first squad
    for player in away_first_squad:
        scorer_list = [player, away_team, row_id, True]
        scorer_list_of_lists.append(scorer_list)

    # Create a list for each player in the home bench
    for player in home_bench:
        scorer_list = [player, home_team, row_id, False]
        scorer_list_of_lists.append(scorer_list)

    # Create a list for each player in the away bench
    for player in away_bench:
        scorer_list = [player, away_team, row_id, False]
        scorer_list_of_lists.append(scorer_list)

enumerated_scorer_list = [[i + 1] + sublist for i, sublist in enumerate(scorer_list_of_lists)]
player_name_to_id = {name: player_id for name, player_id in player_teams_swapped.items()}

# Update the team name in enumerated_scorer_list based on player name
for sublist in enumerated_scorer_list:
    player_name = sublist[1]
    if player_name in player_name_to_id:
        player_id = player_name_to_id[player_name]
        sublist[2] = player_id


        
number_name_dict = {number: list(data.keys())[0] for number, data in list_of_lists}
for sublist in enumerated_scorer_list:
    player_name = sublist[1]
    for key, value in number_name_dict.items():
        if value == player_name:
            sublist[1] = key
            break  # Break out of the loop once the swap is done

insert_statements = [
    f"({squad_id}, {player_id}, {team_id}, {match_id}, {is_starter})"
    for squad_id, player_id, team_id, match_id, is_starter in enumerated_scorer_list
]

big_insert_statement = f"INSERT INTO Squad VALUES {', '.join(insert_statements)};"

print(big_insert_statement)

INSERT INTO Squad VALUES (1, 1, 1, 1, True), (2, 2, 1, 1, True), (3, 3, 1, 1, True), (4, 4, 1, 1, True), (5, 5, 1, 1, True), (6, 6, 1, 1, True), (7, 7, 1, 1, True), (8, 8, 1, 1, True), (9, 9, 1, 1, True), (10, 10, 1, 1, True), (11, 11, 1, 1, True), (12, 12, 17, 1, True), (13, 13, 17, 1, True), (14, 14, 17, 1, True), (15, 15, 17, 1, True), (16, 16, 17, 1, True), (17, 17, 17, 1, True), (18, 18, 17, 1, True), (19, 19, 17, 1, True), (20, 20, 17, 1, True), (21, 21, 17, 1, True), (22, 22, 17, 1, True), (23, 23, 1, 1, False), (24, 24, 1, 1, False), (25, 25, 1, 1, False), (26, 26, 1, 1, False), (27, 27, 1, 1, False), (28, 28, 1, 1, False), (29, 29, 1, 1, False), (30, 30, 1, 1, False), (31, 31, 1, 1, False), (32, 32, 17, 1, False), (33, 33, 17, 1, False), (34, 34, 17, 1, False), (35, 35, 17, 1, False), (36, 36, 18, 1, False), (37, 37, 98, 1, False), (38, 38, 17, 1, False), (39, 39, 17, 1, False), (40, 40, 17, 1, False), (41, 41, 2, 2, True), (42, 42, 2, 2, True), (43, 43, 2, 2, True), (44, 44, 

In [22]:
scorer_dataframe = df[['Home', 'Away', 'Home Scorers', 'Away Scorers', 'Home Penalty Scorers', 'Away Penalty Scorers', 'Home Own Goal Scorers', 'Away Own Goal Scorers', 'Home Goals Minutes', 'Away Goals Minutes', 'Home Penalty Goals Minutes', 'Away Penalty Goals Minutes', 'Home Own Goal Minutes', 'Away Own Goal Minutes']]
goal_data = []

for index, row in scorer_dataframe.iterrows():
    home_scorers = eval(row['Home Scorers']) if pd.notna(row['Home Scorers']) else []
    away_scorers = eval(row['Away Scorers']) if pd.notna(row['Away Scorers']) else []

    # Process normal goals
    for scorer in home_scorers:
        goal_minutes_str = row['Home Goals Minutes']
        if pd.notna(goal_minutes_str):
            goal_minutes_list = ast.literal_eval(goal_minutes_str)
            goal_minutes = goal_minutes_list[home_scorers.index(scorer)].strip("[]'")
        else:
            goal_minutes = None

        goal_data.append([index + 1, scorer, row['Home'], 1, goal_minutes])

    for scorer in away_scorers:
        goal_minutes_str = row['Away Goals Minutes']
        if pd.notna(goal_minutes_str):
            goal_minutes_list = ast.literal_eval(goal_minutes_str)
            goal_minutes = goal_minutes_list[away_scorers.index(scorer)].strip("[]'")
        else:
            goal_minutes = None

        goal_data.append([index + 1, scorer, row['Away'], 1, goal_minutes])

    # Process penalty goals
    home_penalty_scorers = eval(row['Home Penalty Scorers']) if pd.notna(row['Home Penalty Scorers']) else []
    away_penalty_scorers = eval(row['Away Penalty Scorers']) if pd.notna(row['Away Penalty Scorers']) else []

    for scorer in home_penalty_scorers:
        goal_minutes_str = row['Home Penalty Goals Minutes']
        if pd.notna(goal_minutes_str):
            goal_minutes_list = ast.literal_eval(goal_minutes_str)
            goal_minutes = goal_minutes_list[home_penalty_scorers.index(scorer)].strip("[]'")
        else:
            goal_minutes = None

        goal_data.append([index + 1, scorer, row['Home'], 2, goal_minutes])

    for scorer in away_penalty_scorers:
        goal_minutes_str = row['Away Penalty Goals Minutes']
        if pd.notna(goal_minutes_str):
            goal_minutes_list = ast.literal_eval(goal_minutes_str)
            goal_minutes = goal_minutes_list[away_penalty_scorers.index(scorer)].strip("[]'")
        else:
            goal_minutes = None

        goal_data.append([index + 1, scorer, row['Away'], 2, goal_minutes])

    # Process own goals
    home_own_goal_scorers = eval(row['Home Own Goal Scorers']) if pd.notna(row['Home Own Goal Scorers']) else []
    away_own_goal_scorers = eval(row['Away Own Goal Scorers']) if pd.notna(row['Away Own Goal Scorers']) else []

    for scorer in home_own_goal_scorers:
        goal_minutes_str = row['Home Own Goal Minutes']
        if pd.notna(goal_minutes_str):
            goal_minutes_list = ast.literal_eval(goal_minutes_str)
            goal_minutes = goal_minutes_list[home_own_goal_scorers.index(scorer)].strip("[]'")
        else:
            goal_minutes = None

        goal_data.append([index + 1, scorer, row['Home'], 3, goal_minutes])

    for scorer in away_own_goal_scorers:
        goal_minutes_str = row['Away Own Goal Minutes']
        if pd.notna(goal_minutes_str):
            goal_minutes_list = ast.literal_eval(goal_minutes_str)
            goal_minutes = goal_minutes_list[away_own_goal_scorers.index(scorer)].strip("[]'")
        else:
            goal_minutes = None

        goal_data.append([index + 1, scorer, row['Away'], 3, goal_minutes])
# Iterate through each sublist in goal_data
for sublist in goal_data:
    # Get the player name from the sublist
    player_name = sublist[1]
    
    # Check if the player name exists in player_teams_swapped
    if player_name in player_teams_swapped:
        # Replace the team in the sublist with the corresponding number
        sublist[2] = player_teams_swapped[player_name]


player_dict_IDS = {name: number for number, player_info in list_of_lists for name, _ in player_info.items()}

for i in range(len(goal_data)):
    player_name = goal_data[i][1]
    if player_name in player_dict_IDS:
        goal_data[i][1] = player_dict_IDS[player_name]

for i in range(len(goal_data)):
    goal_data[i].insert(0, i + 1)


insert_statement = """INSERT INTO Scorer (ScorerID, MatchID, PlayerID, TeamID, GoalTypeID, Minute) VALUES {};"""
values = ", ".join([f"""({", ".join(map(str, entry))})""" for entry in goal_data])
sql_script = insert_statement.format(values)
sql_script = sql_script.replace("’", "")

print(sql_script)

goal_type = "INSERT INTO GoalType(GoalTypeID, GoalTypeName) VALUES (1, 'Normal'), (2, 'Penalty'), (3, 'Own Goal');"

print(goal_type)


INSERT INTO Scorer (ScorerID, MatchID, PlayerID, TeamID, GoalTypeID, Minute) VALUES (1, 1, 6, 1, 1, 10), (2, 1, 18, 17, 1, 42), (3, 2, 43, 2, 1, 30), (4, 2, 69, 2, 1, 36), (5, 2, 44, 2, 1, 55), (6, 2, 47, 2, 1, 61), (7, 2, 44, 2, 1, 55), (8, 2, 79, 13, 1, 90+3), (9, 3, 84, 3, 1, 7), (10, 3, 98, 16, 1, 12), (11, 4, 122, 62, 1, 22), (12, 6, 209, 18, 1, 37), (13, 6, 231, 18, 1, 78), (14, 6, 234, 18, 1, 87), (15, 6, 213, 18, 1, 90+5), (16, 7, 241, 7, 1, 23), (17, 7, 240, 7, 1, 32), (18, 7, 239, 7, 1, 34), (19, 7, 237, 7, 1, 58), (20, 7, 239, 7, 1, 34), (21, 7, 270, 15, 1, 86), (22, 7, 246, 7, 3, 27), (23, 8, 281, 8, 1, 13), (24, 9, 322, 9, 1, 41), (25, 9, 319, 9, 1, 52), (26, 9, 319, 9, 1, 52), (27, 9, 331, 11, 1, 5), (28, 10, 309, 10, 1, 38), (29, 10, 292, 10, 1, 46), (30, 10, 309, 10, 1, 38), (31, 10, 296, 10, 2, 65), (32, 11, 127, 4, 1, 73), (33, 11, 360, 4, 1, 88), (34, 11, 347, 4, 2, 60), (35, 12, 362, 12, 1, 21), (36, 12, 363, 12, 1, 56), (37, 13, 163, 5, 1, 45), (38, 13, 56, 13, 2, 

In [23]:
match_venues_list = []

for index, row in df.iterrows():
    match_info = [index + 1, row['Venue']]
    match_venues_list.append(match_info)

for match_info in match_venues_list:
    match_info[1] = [key for key, value in venues_dict.items() if value == match_info[1]][0]
    
insert_statement_matchvenue = "INSERT INTO MatchVenue (MatchVenueID, MatchID, VenueID) VALUES "
values_list_matchvenue = []

for index, match_info in enumerate(match_venues_list, start=1):
    match_id, venue_id = match_info
    values_list_matchvenue.append(f"({index}, {match_id}, {venue_id})")

insert_statement_matchvenue += ', '.join(values_list_matchvenue) + ';'

print(insert_statement_matchvenue)

INSERT INTO MatchVenue (MatchVenueID, MatchID, VenueID) VALUES (1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5), (6, 6, 6), (7, 7, 7), (8, 8, 8), (9, 9, 9), (10, 10, 10), (11, 11, 11), (12, 12, 12), (13, 13, 13), (14, 14, 14), (15, 15, 15), (16, 16, 16), (17, 17, 17), (18, 18, 7), (19, 19, 8), (20, 20, 2), (21, 21, 5), (22, 22, 6), (23, 23, 9), (24, 24, 17), (25, 25, 3), (26, 26, 4), (27, 27, 14), (28, 28, 18), (29, 29, 3), (30, 30, 16), (31, 31, 13), (32, 32, 10), (33, 33, 12), (34, 34, 1), (35, 35, 11), (36, 36, 6), (37, 37, 5), (38, 38, 17), (39, 39, 8), (40, 40, 9), (41, 41, 2), (42, 42, 7), (43, 43, 4), (44, 44, 13), (45, 45, 15), (46, 46, 18), (47, 47, 16), (48, 48, 3), (49, 49, 10), (50, 50, 1), (51, 51, 12), (52, 52, 14), (53, 53, 9), (54, 54, 2), (55, 55, 7), (56, 56, 11), (57, 57, 4), (58, 58, 10), (59, 59, 8), (60, 60, 17), (61, 61, 5), (62, 62, 18), (63, 63, 15), (64, 64, 3), (65, 65, 19), (66, 66, 13), (67, 67, 7), (68, 68, 1), (69, 69, 16), (70, 70, 6), (71, 71, 8), 

In [24]:
enum_managers_dict = {manager: idx for idx, manager in enumerate(managers_dict, start=1)}
managers_list = []

for index, row in df.iterrows():
    home_manager_key = enum_managers_dict.get(row['Home Manager'], None)
    away_manager_key = enum_managers_dict.get(row['Away Manager'], None)
    
    if home_manager_key is not None and away_manager_key is not None:
        managers_info = [index + 1, home_manager_key, away_manager_key]
        managers_list.append(managers_info)

insert_statement_matchmanager = "INSERT INTO MatchManager (MatchID, HomeManagerID, AwayManagerID) VALUES "
values_list_matchmanager = []

for match_info in managers_list:
    match_id, home_manager_id, away_manager_id = match_info
    values_list_matchmanager.append(f"({match_id}, {home_manager_id}, {away_manager_id})")

insert_statement_matchmanager += ', '.join(values_list_matchmanager) + ';'

print(insert_statement_matchmanager)

INSERT INTO MatchManager (MatchID, HomeManagerID, AwayManagerID) VALUES (1, 1, 17), (2, 2, 13), (3, 3, 16), (4, 4, 12), (5, 5, 14), (6, 6, 18), (7, 7, 15), (8, 8, 10), (9, 9, 11), (10, 10, 2), (11, 11, 4), (12, 12, 8), (13, 13, 5), (14, 14, 7), (15, 15, 6), (16, 16, 1), (17, 17, 9), (18, 7, 18), (19, 8, 13), (20, 2, 14), (21, 5, 15), (22, 6, 16), (23, 9, 12), (24, 17, 11), (25, 3, 1), (26, 4, 10), (27, 14, 9), (28, 18, 8), (29, 3, 6), (30, 16, 7), (31, 13, 4), (32, 10, 17), (33, 12, 11), (34, 1, 5), (35, 11, 13), (36, 6, 1), (37, 5, 18), (38, 17, 12), (39, 8, 14), (40, 9, 10), (41, 2, 16), (42, 7, 3), (43, 4, 15), (44, 13, 17), (45, 15, 9), (46, 18, 4), (47, 16, 8), (48, 3, 5), (49, 10, 11), (50, 1, 7), (51, 12, 2), (52, 14, 6), (53, 9, 13), (54, 2, 18), (55, 7, 6), (56, 11, 14), (57, 4, 1), (58, 10, 12), (59, 8, 3), (60, 17, 15), (61, 5, 16), (62, 18, 9), (63, 15, 11), (64, 3, 4), (65, 14, 10), (66, 13, 12), (67, 7, 8), (68, 1, 2), (69, 16, 17), (70, 6, 5), (71, 8, 6), (73, 4, 14), (7

In [25]:
swap_dict = {v[0]: k for k, v in captains_enum_dict.items()}

df['Away Captain'] = df['Away Captain'].map(swap_dict)
df['Home Captain'] = df['Home Captain'].map(swap_dict)

insert_statement_matchcaptain = "INSERT INTO MatchCaptain (MatchID, HomeCaptainID, AwayCaptainID) VALUES "
values_list_matchcaptain = []

for index, row in df.iterrows():
    match_id = index + 1
    home_captain_id = row['Home Captain']
    away_captain_id = row['Away Captain']
    values_list_matchcaptain.append(f"({match_id}, {home_captain_id}, {away_captain_id})")

insert_statement_matchcaptain += ', '.join(values_list_matchcaptain) + ';'

print(insert_statement_matchcaptain)


INSERT INTO MatchCaptain (MatchID, HomeCaptainID, AwayCaptainID) VALUES (1, 1, 46), (2, 3, 36), (3, 6, 43), (4, 10, 34), (5, 12, 38), (6, 16, 49), (7, 19, 40), (8, 22, 30), (9, 24, 32), (10, 27, 3), (11, 31, 10), (12, 34, 22), (13, 36, 12), (14, 38, 19), (15, 39, 17), (16, 43, 1), (17, 46, 24), (18, 19, 49), (19, 22, 36), (20, 3, 38), (21, 12, 39), (22, 17, 43), (23, 24, 34), (24, 46, 31), (25, 6, 1), (26, 10, 28), (27, 38, 24), (28, 49, 22), (29, 6, 16), (30, 43, 19), (31, 36, 10), (32, 28, 46), (33, 34, 31), (34, 1, 12), (35, 31, 36), (36, 16, 1), (37, 12, 49), (38, 46, 34), (39, 23, 38), (40, 24, 28), (41, 3, 43), (42, 19, 7), (43, 10, 39), (44, 36, 46), (45, 39, 24), (46, 49, 10), (47, 43, 23), (48, 6, 12), (49, 28, 32), (50, 1, 21), (51, 34, 3), (52, 38, 16), (53, 24, 36), (54, 3, 49), (55, 19, 16), (56, 31, 38), (57, 10, 1), (58, 28, 34), (59, 23, 7), (60, 46, 39), (61, 12, 43), (62, 49, 26), (63, 40, 31), (64, 7, 10), (65, 38, 28), (66, 36, 34), (67, 19, 22), (68, 1, 3), (69, 43

In [26]:
rev_referees_dict = {v: k for k, v in referees_dict.items()}
rev_ar1_dict = {v: k for k, v in ar1_dict.items()}
rev_ar2_dict = {v: k for k, v in ar2_dict.items()}
rev_fourth_official_dict = {v: k for k, v in fourth_official_dict.items()}
rev_var_dict = {v: k for k, v in var_dict.items()}

df['Referee'] = df['Referee'].map(rev_referees_dict)
df['AR1'] = df['AR1'].map(rev_ar1_dict)
df['AR2'] = df['AR2'].map(rev_ar2_dict)
df['Fourth Official'] = df['Fourth Official'].map(rev_fourth_official_dict)
df['VAR'] = df['VAR'].map(rev_var_dict)

In [27]:
insert_statement_matchreferee = "INSERT INTO MatchReferee (MatchID, RefereeID, AR1ID, AR2ID, FourthOfficialID, VARID) VALUES "
values_list_matchreferee = []

for index, row in df.iterrows():
    match_id = index + 1
    referee_id = row['Referee']
    ar1_id = row['AR1']
    ar2_id = row['AR2']
    fourth_official_id = row['Fourth Official']
    var_id = row['VAR']
    values_list_matchreferee.append(f"({match_id}, {referee_id}, {ar1_id}, {ar2_id}, {fourth_official_id}, {var_id})")

insert_statement_matchreferee += ', '.join(values_list_matchreferee) + ';'

print(insert_statement_matchreferee)


INSERT INTO MatchReferee (MatchID, RefereeID, AR1ID, AR2ID, FourthOfficialID, VARID) VALUES (1, 1, 1, 1, 1, 1), (2, 2, 2, 2, 2, 2), (3, 3, 3, 3, 3, 3), (4, 4, 4, 4, 4, 4), (5, 5, 5, 5, 5, 5), (6, 6, 6, 6, 6, 6), (7, 7, 7, 7, 7, 7), (8, 8, 8, 8, 8, 3), (9, 9, 9, 9, 9, 8), (10, 10, 10, 10, 10, 5), (11, 11, 11, 11, 11, 9), (12, 12, 12, 12, 12, 5), (13, 13, 13, 9, 13, 10), (14, 14, 14, 1, 14, 11), (15, 15, 15, 13, 15, 12), (16, 16, 16, 14, 16, 7), (17, 17, 17, 15, 17, 12), (18, 2, 18, 14, 18, 13), (19, 3, 19, 10, 7, 14), (20, 4, 4, 16, 19, 15), (21, 18, 20, 17, 20, 5), (22, 19, 9, 18, 21, 2), (23, 20, 21, 19, 22, 7), (24, 5, 5, 5, 23, 16), (25, 6, 22, 20, 5, 4), (26, 7, 7, 7, 15, 17), (27, 15, 15, 13, 23, 4), (28, 1, 11, 1, 2, 12), (29, 17, 17, 4, 24, 18), (30, 8, 8, 17, 25, 5), (31, 9, 9, 21, 9, 19), (32, 16, 16, 14, 26, 20), (33, 14, 14, 18, 27, 21), (34, 21, 23, 22, 18, 22), (35, 7, 24, 4, 14, 13), (36, 10, 10, 10, 28, 11), (37, 12, 12, 15, 29, 16), (38, 22, 25, 11, 30, 8), (39, 19, 9, 

In [28]:
insert_statement_match_formation = "INSERT INTO MatchFormation (MatchID, HomeFormationID, AwayFormationID) VALUES "
insert_statement_match_Attendance = "INSERT INTO Attendance (MatchID, Attendance) VALUES "
rev_formations_dict = {v: k for k, v in formations_dict.items()}
df['Home Formation'] = df['Home Formation'].map(rev_formations_dict)
df['Away Formation'] = df['Away Formation'].map(rev_formations_dict)

values_list_match_formation =[]
for index, row in df.iterrows():
    match_id = index + 1
    home_match_formation = row['Home Formation']
    away_match_formation = row['Away Formation']
    values_list_match_formation.append(f"({match_id}, {home_match_formation}, {away_match_formation})")

insert_statement_match_formation += ', '.join(values_list_match_formation) + ';'

print(insert_statement_match_formation)


values_list_match_attendance =[]
for index, row in df.iterrows():
    match_id = index + 1
    attandance = 'NULL' if pd.isnull(row['Attendance']) else int(row['Attendance'])

    values_list_match_attendance.append(f"({match_id}, {attandance})")

insert_statement_match_Attendance += ', '.join(values_list_match_attendance) + ';'

print(insert_statement_match_Attendance)


INSERT INTO MatchFormation (MatchID, HomeFormationID, AwayFormationID) VALUES (1, 1, 1), (2, 2, 7), (3, 3, 1), (4, 1, 9), (5, 4, 2), (6, 1, 1), (7, 1, 2), (8, 5, 1), (9, 4, 9), (10, 1, 2), (11, 1, 1), (12, 1, 7), (13, 4, 4), (14, 6, 4), (15, 1, 1), (16, 1, 1), (17, 1, 8), (18, 4, 1), (19, 7, 4), (20, 2, 6), (21, 6, 1), (22, 1, 1), (23, 8, 9), (24, 1, 1), (25, 7, 1), (26, 1, 1), (27, 6, 8), (28, 1, 7), (29, 7, 3), (30, 1, 21), (31, 6, 1), (32, 1, 1), (33, 1, 2), (34, 2, 9), (35, 2, 10), (36, 7, 1), (37, 6, 1), (38, 1, 1), (39, 7, 2), (40, 8, 1), (41, 5, 1), (42, 9, 5), (43, 1, 1), (44, 9, 2), (45, 1, 4), (46, 10, 1), (47, 1, 7), (48, 7, 10), (49, 2, 2), (50, 2, 7), (51, 1, 14), (52, 2, 7), (53, 8, 8), (54, 11, 9), (55, 1, 5), (56, 2, 2), (57, 1, 7), (58, 2, 1), (59, 2, 5), (60, 1, 2), (61, 6, 1), (62, 5, 8), (63, 5, 1), (64, 7, 1), (65, 2, 9), (66, 12, 9), (67, 9, 5), (68, 2, 11), (69, 1, 1), (70, 7, 12), (71, 7, 5), (72, 1, 2), (73, 9, 2), (74, 2, 1), (75, 5, 2), (76, 1, 5), (77, 4, 1)

In [29]:
with open('Big_Five_SQL_Data.txt', 'w') as file:
    print(sql_insert_laegue, file=file)
    print(goal_type, file=file)
    print(sql_insert_teams, file=file)
    print(sql_insert_managers, file=file)
    print(sql_insert_captains, file=file)
    print(sql_insert_formations, file=file)
    print(sql_insert_referees, file=file)
    print(sql_insert_ar1, file=file)
    print(sql_insert_ar2, file=file)
    print(sql_insert_fourth_official, file=file)
    print(sql_insert_var, file=file)
    print(sql_insert_venues, file=file)
    print(sql_insert_match_dates, file=file)
    print(final_sql_statement_matchdata, file=file)
    print(insert_statement_players, file=file)
    print(big_insert_statement, file=file) 
    print(sql_script, file=file)
    print(insert_statement_matchvenue, file=file)
    print(insert_statement_matchmanager, file=file)
    print(insert_statement_matchcaptain, file=file)
    print(insert_statement_matchreferee, file=file)
    print(insert_statement_match_formation, file=file)
    print(insert_statement_match_Attendance, file=file)


In [30]:
# Define the table names and their corresponding columns in the dataframe
tables = {
    'FoulStatistics': ['Home Fouls', 'Away Fouls'],
    'CornerStatistics': ['Home Corners', 'Away Corners'],
    'CrossStatistics': ['Home Crosses', 'Away Crosses'],
    'TouchStatistics': ['Home Touches', 'Away Touches'],
    'TackleStatistics': ['Home Tackles', 'Away Tackles'],
    'AerialStatistics': ['Home Aerials Won', 'Away Aerials Won'],
    'ClearanceStatistics': ['Home Clearances', 'Away Clearances'],
    'OffsideStatistics': ['Home Offsides', 'Away Offsides'],
    'GoalKickStatistics': ['Home Goal Kicks', 'Away Goal Kicks'],
    'ThrowInStatistics': ['Home Throw Ins', 'Away Throw Ins'],
    'LongBallStatistics': ['Home Long Balls', 'Away Long Balls'],
    'YellowCards': ['Home Yellow Cards', 'Away Yellow Cards'],
    'RedCards': ['Home Red Cards', 'Away Red Cards']
}

with open('Big_Five_SQL_Data.txt', 'a+') as file:
    for table, columns in tables.items():
        home_insert = str(columns[0])
        away_insert = str(columns[1])
        home_insert = home_insert.replace(" ", "")
        away_insert = away_insert.replace(" ", "")
        insert_statement = f"INSERT INTO {table} (MatchID, {home_insert}, {away_insert}) VALUES "
        values_list = []

        for index, row in df.iterrows():
            match_id = index + 1
            home_stat = 'NULL' if pd.isnull(row[columns[0]]) else row[columns[0]]
            away_stat = 'NULL' if pd.isnull(row[columns[1]]) else row[columns[1]]
            values_list.append(f"({match_id}, {home_stat}, {away_stat})")

        insert_statement += ', '.join(values_list) + ';\n'
        # Write final data to the file
        file.write(insert_statement)
